In [ ]:
import urllib
import pandas as pd
from datetime import datetime
import zipfile
from io import StringIO
pd.set_option('display.max_rows', 500)

In [ ]:
data = pd.DataFrame()

In [ ]:
endPoint = 'https://data.binance.vision/data/spot/daily/trades/'
instrument = 'BTCUSDT'
fileName_2022_03_01 = '/BTCUSDT-trades-2022-03-01.zip'
fileName_2022_03_02 = '/BTCUSDT-trades-2022-03-02.zip'
fileName_2022_03_03 = '/BTCUSDT-trades-2022-03-03.zip'

In [ ]:
file_2022_03_01 =  urllib.request.urlretrieve(endPoint + instrument + fileName_2022_03_01, fileName_2022_03_01)
file_2022_03_02 =  urllib.request.urlretrieve(endPoint + instrument + fileName_2022_03_02, fileName_2022_03_02)
file_2022_03_03 =  urllib.request.urlretrieve(endPoint + instrument + fileName_2022_03_03, fileName_2022_03_03)

In [ ]:
for trade_data in [file_2022_03_01,file_2022_03_02, file_2022_03_03]:
    zip_file_object = zipfile.ZipFile(trade_data[0], 'r')
    first_file = zip_file_object.namelist()[0]
    file = zip_file_object.open(first_file)
    content = file.read() #unzipped files are in bytes format at this point
    #to convert data from bytes type to a pandas dataframe                                  
    string_format=str(content,'utf-8')
    unzippedData = StringIO(string_format) 
    csvConverted = pd.read_csv(unzippedData,names =['trade_id','tradedPrice','tradedSize','tradedNotional','timeStamp','isBuyerMaker','isBestMatch'])
    data = pd.concat([data,csvConverted]) 
data = data.reset_index(drop = True)
data = data.drop(['trade_id','isBuyerMaker','isBestMatch'],axis = 1) #to drop the columns that are not required

In [ ]:
tradesheet = pd.DataFrame(columns = ['type','spot','buy','buy_timeStamp','sell','sell_timeStamp','pnl'])

In [ ]:
bid_margin = 0.0003 
ask_margin = 0.0003
balance = 1000 #The nominal value(USDT) for each trade, this will be used to convert to BTC for actual trade quantity
buy_position = False # if the position(buy or sell) is false, it means that the position is open and not executed yet
sell_position = False
opening_time = data.loc[0,'timeStamp']
opening_spot = data.loc[0,'tradedPrice'] # opening spot price of BTCUSDT
buy_order = opening_spot * (1-bid_margin) 
sell_order = opening_spot * (1+ask_margin)
trade_id = 0 #counts the number of trades and also matches a buy order to a sell order
refresh_timeStamp = opening_time
carry_buy = {} #A dictionary that holds all the open buy trades that did not execute within the refresh time, so they will be carried forward to the next refresh rate
carry_sell = {}#A dictionary that holds all the open sell trades that did not execute within the refresh time, so they will be carried forward to the next refresh rate
carry_buy[buy_order] = trade_id #matching a buy order to it's trade id
carry_sell[sell_order] = trade_id #matching a sell order to it's trade id
refresh_time = 360000 # quote refresh time in milliseconds(6 minutes)

In [ ]:
for i in data.index:
    #Since there will a latency of 100ms between the algorithm plaacing the order and the order reaching the exchange, any order matches that happen in between this time wont be considered as executable trades
    if (data.loc[i,'timeStamp'] + 100 > refresh_timeStamp) and (data.loc[i,'timeStamp'] < refresh_timeStamp + refresh_time):
        if (buy_position == False) and (sell_position == False):
            if data.loc[i,'tradedPrice'] > sell_order and (len(carry_sell) < 6):
                sell = (balance/data.loc[i,'tradedPrice']) #converting USDT to BTC
                sell_position = True
                tradesheet.loc[trade_id,'spot'] = data.loc[i,'tradedPrice']
                tradesheet.loc[trade_id,'sell'] = sell
                tradesheet.loc[trade_id,'sell_timeStamp'] = data.loc[i,'timeStamp']
                tradesheet.loc[trade_id,'type'] = 1
                continue
            elif data.loc[i,'tradedPrice'] < buy_order and (len(carry_buy) < 6):
                buy = (balance/data.loc[i,'tradedPrice'])  #converting USDT to BTC
                buy_position = True
                tradesheet.loc[trade_id,'spot'] = data.loc[i,'tradedPrice']
                tradesheet.loc[trade_id,'buy'] = buy
                tradesheet.loc[trade_id,'buy_timeStamp'] = data.loc[i,'timeStamp']
                tradesheet.loc[trade_id,'type'] = 0

                continue
        elif (buy_position == False) and (sell_position == True) and (len(carry_sell) < 6):
            if data.loc[i,'tradedPrice'] < buy_order:
                buy = (balance/data.loc[i,'tradedPrice'])
                buy_position = True
                tradesheet.loc[trade_id,'buy'] = buy
                tradesheet.loc[trade_id,'buy_timeStamp'] = data.loc[i,'timeStamp']

                continue
        elif (buy_position == True) and (sell_position == False) and (len(carry_buy) < 6):
            if data.loc[i,'tradedPrice'] > sell_order:
                sell = (balance/data.loc[i,'tradedPrice'])
                sell_position = True
                tradesheet.loc[trade_id,'sell'] = sell
                tradesheet.loc[trade_id,'sell_timeStamp'] = data.loc[i,'timeStamp']
                continue
        elif(buy_position == True) and (sell_position == True):
            #if both trades are executed don't create more trades until the next refresh time
            pass
        
        #checking for old trades
        for values in list(carry_buy.keys()):
            if data.loc[i,'tradedPrice'] < values:
                tradesheet.loc[carry_buy[values],'buy'] = (balance/data.loc[i,'tradedPrice'])
                tradesheet.loc[carry_buy[values],'buy_timeStamp'] =data.loc[i,'timeStamp']
                carry_buy.pop(values)
            else:
                continue
        for values in list(carry_sell.keys()):
            if data.loc[i,'tradedPrice'] > values:
                tradesheet.loc[carry_sell[values],'sell'] = (balance/data.loc[i,'tradedPrice'])
                tradesheet.loc[carry_sell[values],'sell_timeStamp'] = data.loc[i,'timeStamp']
                carry_sell.pop(values)
            else:
                continue
    elif data.loc[i,'timeStamp'] > refresh_timeStamp + refresh_time:
        refresh_timeStamp = data.loc[i,'timeStamp'] 

        # if both the trades are not executed, cancel both trades
        if (buy_position == False) and (sell_position == False):
            try:
                carry_buy.pop(buy_order)
                carry_sell.pop(sell_order)
            except Exception as e:
                print(e)
        trade_id += 1
        buy_position = False
        sell_position = False
        refresh_spot = data.loc[i,'tradedPrice']
        buy_order = refresh_spot * (1-bid_margin)
        carry_buy[buy_order] = trade_id
        sell_order = refresh_spot * (1+ask_margin)
        carry_sell[sell_order] = trade_id
        if len(carry_sell) > 5:
            exit_id = sorted(carry_sell.values())[0]
            tradesheet.loc[exit_id,'sell'] = (balance/refresh_spot)
            tradesheet.loc[exit_id,'sell_timeStamp'] = refresh_timeStamp
            for key,value in carry_sell.items():
                if value == exit_id:
                    carry_sell.pop(key)
                    break
        if len(carry_buy) > 5:
            exit_id = sorted(carry_buy.values())[0]
            tradesheet.loc[exit_id,'buy'] = (balance/refresh_spot)
            tradesheet.loc[exit_id,'buy_timeStamp'] = refresh_timeStamp
            for key,value in carry_buy.items():
                if value == exit_id:
                    carry_buy.pop(key)
                    break
            

In [ ]:
tradesheet = tradesheet[tradesheet['sell'].notna()]
tradesheet = tradesheet[tradesheet['buy'].notna()]

In [ ]:
tradesheet.pnl = (tradesheet.buy - tradesheet.sell) * tradesheet.spot

In [ ]:
tradesheet_01_03_2022 = tradesheet[tradesheet.buy_timeStamp < 1646092800000 + 86400000]
tradesheet_03_03_2022 = tradesheet[tradesheet.buy_timeStamp > 1646092800000 + 2*(86400000)]
tradesheet_02_03_2022 = tradesheet.loc[tradesheet_01_03_2022.index[-1]:tradesheet_03_03_2022.index[0]]
tradesheet_02_03_2022 = tradesheet_02_03_2022.reset_index()
tradesheet_03_03_2022 = tradesheet_03_03_2022.reset_index()
tradesheet_01_03_2022 = tradesheet_01_03_2022.reset_index()

In [ ]:
metrics =pd.DataFrame(columns = ['date','No_of_trades','No_of_winners','No_of_losers','Avg_winner','Avg_loser','PnL_basisPoints','PnL_gross','Max_drawdown'])

In [ ]:
metrics.loc[0,'date'] = '01/03/2022'
metrics.loc[0,'No_of_trades'] = len(tradesheet_01_03_2022)
metrics.loc[0,'No_of_winners'] = len(tradesheet_01_03_2022[tradesheet_01_03_2022.pnl > 0])
metrics.loc[0,'No_of_losers'] = len(tradesheet_01_03_2022[tradesheet_01_03_2022.pnl < 0])
metrics.loc[0,'Avg_winner'] = tradesheet_01_03_2022[tradesheet_01_03_2022.pnl > 0].pnl.sum()/metrics.loc[0,'No_of_winners']
metrics.loc[0,'Avg_loser'] = tradesheet_01_03_2022[tradesheet_01_03_2022.pnl < 0].pnl.sum()/metrics.loc[0,'No_of_losers']
metrics.loc[0,'PnL_gross' ] = tradesheet_01_03_2022.pnl.sum()
metrics.loc[0,'PnL_basisPoints'] = (metrics.loc[0,'PnL_gross']/5000)*10000

min_balance = 1000
opening_balance = 1000
for i in range(len(tradesheet_01_03_2022)-1):
    opening_balance += tradesheet_01_03_2022.loc[i,'pnl'] 
    if opening_balance < min_balance:
        min_balance = opening_balance
metrics.loc[0,'Max_drawdown'] = 1000 - min_balance

In [ ]:
metrics.loc[1,'date'] = '02/03/2022'
metrics.loc[1,'No_of_trades'] = len(tradesheet_02_03_2022)
metrics.loc[1,'No_of_winners'] = len(tradesheet_02_03_2022[tradesheet_02_03_2022.pnl > 0])
metrics.loc[1,'No_of_losers'] = len(tradesheet_02_03_2022[tradesheet_02_03_2022.pnl < 0])
metrics.loc[1,'Avg_winner'] = tradesheet_02_03_2022[tradesheet_02_03_2022.pnl > 0].pnl.sum()/metrics.loc[1,'No_of_winners']
metrics.loc[1,'Avg_loser'] = tradesheet_02_03_2022[tradesheet_02_03_2022.pnl < 0].pnl.sum()/metrics.loc[1,'No_of_losers']
metrics.loc[1,'PnL_gross' ] = tradesheet_02_03_2022.pnl.sum()
metrics.loc[1,'PnL_basisPoints'] = (metrics.loc[1,'PnL_gross']/5000)*10000

min_balance = 1000
opening_balance = 1000
for i in range(len(tradesheet_02_03_2022)-1):
    opening_balance += tradesheet_02_03_2022.loc[i,'pnl'] 
    if opening_balance < min_balance:
        min_balance = opening_balance
metrics.loc[1,'Max_drawdown'] = 1000 - min_balance

In [ ]:
metrics.loc[2,'date'] = '03/03/2022'
metrics.loc[2,'No_of_trades'] = len(tradesheet_03_03_2022)
metrics.loc[2,'No_of_winners'] = len(tradesheet_03_03_2022[tradesheet_03_03_2022.pnl > 0])
metrics.loc[2,'No_of_losers'] = len(tradesheet_03_03_2022[tradesheet_03_03_2022.pnl < 0])
metrics.loc[2,'Avg_winner'] = tradesheet_03_03_2022[tradesheet_03_03_2022.pnl > 0].pnl.sum()/metrics.loc[2,'No_of_winners']
metrics.loc[2,'Avg_loser'] = tradesheet_03_03_2022[tradesheet_03_03_2022.pnl < 0].pnl.sum()/metrics.loc[2,'No_of_losers']
metrics.loc[2,'PnL_gross' ] = tradesheet_03_03_2022.pnl.sum()
metrics.loc[2,'PnL_basisPoints'] = (metrics.loc[2,'PnL_gross']/5000)*10000

min_balance = 1000
opening_balance = 1000
for i in range(len(tradesheet_03_03_2022)-1):
    opening_balance += tradesheet_03_03_2022.loc[i,'pnl'] 
    if opening_balance < min_balance:
        min_balance = opening_balance
metrics.loc[2,'Max_drawdown'] = 1000 - min_balance

In [ ]:
metrics